In [321]:
import requests
import pandas as pd

years = range(2005, 2024)  # From 2010 to 2022
data = []

# Replace with your API key
API_KEY = '59951fa51ae0813c57d78e2f22413294efb566a6'

# DP03_0003E, DP03_0005E
for year in years:
    if year > 2010:
        # For years greater than 2010, make a single API call to get all data
        if(year != 2020):
            url = f"https://api.census.gov/data/{year}/acs/acs1?get=NAME,B01001_001E,B19013_001E,B23025_005E,B23025_003E,B01002_001E,B07001_001E&for=place:*&key={API_KEY}"
        else:
            url = f"https://api.census.gov/data/{year}/acs/acs5?get=NAME,B01001_001E,B19013_001E,B23025_005E,B23025_003E,B01002_001E,B07001_001E&for=place:*&key={API_KEY}"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch data for year {year}")
            continue
        json_data = response.json()
        for row in json_data[1:]:  # Skip the header row
            city_name = row[0]
            try:
                population = int(row[1])
                mean_income = float(row[2])
                unemployed = float(row[3])
                total_labor_force = float(row[4])
                median_age = float(row[5])
                geo_mobility = float(row[6])
            except:
                population = 0

            # Filter for population greater than 100,000
            if population > 100000:
                data.append({
                    'Year': year,
                    'City': city_name,
                    'Total Population': population,
                    'Mean Household Income': mean_income,
                    'Unemployment Rate': unemployed/total_labor_force*100,
                    'Median Age': median_age,
                    'Geographic Mobility': geo_mobility,
                })
    else:
        # For years <= 2010, make two API calls: one for general data and one for unemployment data
        url_general = f"https://api.census.gov/data/{year}/acs/acs1?get=NAME,B01001_001E,B19013_001E,B01002_001E,B07001_001E&for=place:*&key={API_KEY}"
        url_unemployment = f"https://api.census.gov/data/{year}/acs/acs1/profile?get=NAME,DP03_0003E,DP03_0005E&for=place:*&key={API_KEY}"
        
        response_general = requests.get(url_general)
        response_unemployment = requests.get(url_unemployment)
        
        if response_general.status_code != 200 or response_unemployment.status_code != 200:
            print(f"Failed to fetch data for year {year}")
            continue
        
        json_data_general = response_general.json()
        json_data_unemployment = response_unemployment.json()
        
        # Create a lookup for unemployment data by city name
        unemployment_dict = {}
        for row in json_data_unemployment[1:]:  # Skip the header row
            city_name = row[0]
            try:
                unemployed = float(row[1])
                total_labor_force = float(row[2])
                unemployment_dict[city_name] = (unemployed, total_labor_force)
            except:
                unemployment_dict[city_name] = (0, 0)
        
        for row in json_data_general[1:]:  # Skip the header row
            city_name = row[0]
            try:
                population = int(row[1])
                mean_income = float(row[2])
                median_age = float(row[3])
                geo_mobility = float(row[4])
                unemployed, total_labor_force = unemployment_dict.get(city_name, (0, 0))
            except:
                population = 0

            # Filter for population greater than 100,000
            if population > 100000:
                if total_labor_force > 0:  # Prevent division by zero
                    unemployment_rate = unemployed / total_labor_force 
                else:
                    unemployment_rate = None
                
                data.append({
                    'Year': year,
                    'City': city_name,
                    'Total Population': population,
                    'Mean Household Income': mean_income,
                    'Unemployment Rate': unemployment_rate,
                    'Median Age': median_age,
                    'Geographic Mobility': geo_mobility,
                })

# Create a DataFrame
df = pd.DataFrame(data)
state_abbreviation_map = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI',
    'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Step 2: Replace state names with abbreviations in the 'City' column
def replace_state_names(city_name):
    for state, abbrev in state_abbreviation_map.items():
        if state in city_name:
            return city_name.replace(state, abbrev)
    return city_name
pivot_income = df.pivot(index='City', columns='Year', values='Mean Household Income')
df_income = pivot_income
df_income = df_income.dropna()
# cleaning out stuff
df_income.reset_index(inplace=True)
df_income = df_income[~df_income['City'].str.contains("Puerto Rico", case=False, na=False)]
df_income['City'] = df_income['City'].str.replace(r' \bcity\b', '', regex=True)
df_income['City'] = df_income['City'].str.replace(r' \bmunicipality\b', '', regex=True)
df_income['City'] = df_income['City'].str.replace(r' \bCDP\b', '', regex=True)
df_income['City'] = df_income['City'].str.strip()
df_income['City'] = df_income['City'].apply(replace_state_names)
df_income.reset_index(inplace=True)
df_income = df_income.drop(columns=['index'], errors='ignore')
df_income


Year,City,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,"Albuquerque, NM",41820.0,43021.0,43677.0,46437.0,44594.0,46079.0,43606.0,45373.0,48357.0,46433.0,47096.0,50522.0,50456.0,51099.0,55567.0,53936.0,58512.0,64757.0,67907.0
1,"Anaheim, CA",52158.0,55720.0,57059.0,61128.0,55154.0,54157.0,56858.0,55464.0,57550.0,62198.0,63104.0,64464.0,70395.0,76154.0,76075.0,76723.0,81747.0,85133.0,84872.0
2,"Anchorage, AK",61217.0,63656.0,68726.0,75637.0,72832.0,72128.0,72813.0,72575.0,79045.0,76337.0,78662.0,85634.0,79166.0,83648.0,82716.0,84813.0,86654.0,100751.0,94437.0
3,"Arlington, VA",80433.0,87350.0,94876.0,101171.0,96218.0,94986.0,100735.0,100474.0,102501.0,109266.0,106768.0,110388.0,117237.0,122394.0,119755.0,122604.0,125651.0,132380.0,140219.0
4,"Arlington, TX",48992.0,50582.0,50447.0,52777.0,50938.0,48752.0,50655.0,51285.0,51400.0,52141.0,53487.0,54272.0,57083.0,63091.0,61716.0,63351.0,59889.0,70433.0,69208.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,"Tulsa, OK",35966.0,36137.0,38270.0,39657.0,38426.0,38220.0,40123.0,40440.0,41495.0,42803.0,43322.0,43134.0,45894.0,47583.0,49158.0,49474.0,51008.0,54040.0,56821.0
83,"Vancouver, WA",40743.0,45884.0,47989.0,50990.0,45701.0,47307.0,48909.0,45718.0,47379.0,51478.0,51988.0,56615.0,61079.0,60988.0,66679.0,63617.0,69993.0,76799.0,80618.0
84,"VA Beach, VA",58545.0,61333.0,61462.0,65776.0,59298.0,64212.0,64614.0,61626.0,62855.0,68816.0,67281.0,71117.0,72586.0,77059.0,79054.0,78136.0,81634.0,83245.0,91141.0
85,"Washington, DC",47221.0,51847.0,54317.0,57936.0,59290.0,60903.0,63124.0,66583.0,67572.0,71648.0,75628.0,75506.0,82372.0,85203.0,92266.0,90842.0,90088.0,101027.0,108210.0


In [322]:
pivot_unem = df.pivot(index='City', columns='Year', values='Unemployment Rate')
df_unem = pivot_unem
df_unem = df_unem.dropna()
# cleaning out stuff
df_unem.reset_index(inplace=True)
df_unem = df_unem[~df_unem['City'].str.contains("Puerto Rico", case=False, na=False)]
df_unem['City'] = df_unem['City'].str.replace(r' \bcity\b', '', regex=True)
df_unem['City'] = df_unem['City'].str.replace(r' \bmunicipality\b', '', regex=True)
df_unem['City'] = df_unem['City'].str.replace(r' \bCDP\b', '', regex=True)
df_unem['City'] = df_unem['City'].str.strip()
df_unem['City'] = df_unem['City'].apply(replace_state_names)
df_unem.reset_index(inplace=True)
df_unem = df_unem.drop(columns=['index'], errors='ignore')
df_unem

Year,City,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,"Albuquerque, NM",14.533702,18.786019,19.911130,19.150972,13.877965,12.436434,10.542100,8.817330,7.260191,8.308843,6.681222,5.212342,4.805098,5.734425,5.082790,5.594741,6.105605,4.188383,4.337847
1,"Anaheim, CA",14.052579,17.116733,15.048664,11.264549,8.316300,6.947624,11.514510,11.795449,9.850070,6.379095,6.688202,5.687834,4.870972,4.212351,3.736232,5.253194,7.739288,3.760461,4.800021
2,"Anchorage, AK",12.813669,12.609018,14.652567,16.511551,13.173254,11.754007,7.661335,6.494628,6.606993,5.718332,6.380028,5.161294,5.820640,5.451503,5.422420,5.234905,6.645122,4.071370,4.253729
3,"Arlington, VA",31.389930,49.631138,39.573277,42.709646,17.357022,23.814249,3.375747,4.198722,3.452558,3.231800,4.125580,2.790432,2.515723,2.024391,2.240866,2.845019,3.427225,1.953504,3.585866
4,"Arlington, TX",11.645064,13.063362,16.550008,16.860931,11.282053,8.792764,9.248439,8.889318,7.271395,6.764301,4.317417,4.675709,4.355157,3.586946,5.522196,5.207499,6.383448,4.230644,3.797428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,"Tulsa, OK",14.075982,14.096175,16.782754,21.912674,13.843573,10.640480,8.010352,7.862066,7.173986,5.581463,6.377931,7.008047,6.854466,5.031363,5.547656,6.213191,7.054247,4.833533,6.023028
83,"Vancouver, WA",11.566898,12.620030,13.386219,12.329842,6.921437,6.491887,11.518850,11.966959,9.433639,9.233544,5.439369,5.538916,6.108281,4.402471,4.667593,5.034493,7.572812,6.928904,4.225046
84,"VA Beach, VA",20.551649,25.859625,27.103135,20.557803,15.339974,15.152082,6.348781,7.094098,6.254390,5.334866,5.622950,4.695007,3.980565,4.218738,4.146352,4.379281,4.135830,3.412326,4.105642
85,"Washington, DC",10.958860,11.788686,12.381388,12.881559,9.013277,8.060211,11.377442,10.342908,10.446569,8.859504,7.250638,6.919595,6.590778,7.541190,6.263807,7.119626,8.200761,5.112622,5.258939


In [323]:
pivot_age = df.pivot(index='City', columns='Year', values='Median Age')
df_age = pivot_age
df_age = df_age.dropna()
# cleaning out stuff
df_age.reset_index(inplace=True)
df_age = df_age[~df_age['City'].str.contains("Puerto Rico", case=False, na=False)]
df_age['City'] = df_age['City'].str.replace(r' \bcity\b', '', regex=True)
df_age['City'] = df_age['City'].str.replace(r' \bmunicipality\b', '', regex=True)
df_age['City'] = df_age['City'].str.replace(r' \bCDP\b', '', regex=True)
df_age['City'] = df_age['City'].str.strip()
df_age['City'] = df_age['City'].apply(replace_state_names)
df_age.reset_index(inplace=True)
df_age = df_age.drop(columns=['index'], errors='ignore')
df_age

Year,City,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,"Albuquerque, NM",35.8,35.3,35.2,35.5,34.0,35.2,35.5,35.3,35.8,35.7,36.0,36.5,37.0,37.2,37.3,37.1,38.9,38.4,39.2
1,"Anaheim, CA",30.0,32.2,32.5,31.9,32.2,32.7,32.1,33.2,33.8,34.2,33.6,34.1,34.3,34.4,35.2,34.8,37.2,36.0,37.1
2,"Anchorage, AK",33.8,33.5,33.3,33.4,32.3,32.7,32.9,33.0,32.4,32.1,32.2,33.0,34.0,34.3,34.2,33.9,35.2,35.4,36.0
3,"Arlington, VA",36.7,37.7,37.2,38.0,33.2,33.5,33.9,33.6,33.8,34.0,34.4,34.8,34.8,34.7,35.0,34.8,35.9,35.4,35.7
4,"Arlington, TX",31.6,31.6,31.4,32.0,31.4,32.0,31.2,31.9,32.4,33.2,32.6,32.4,33.1,33.1,33.6,32.9,34.7,32.8,34.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,"Tulsa, OK",35.3,36.1,35.6,35.8,34.9,35.1,34.8,34.8,34.7,35.4,35.0,34.5,35.5,35.4,35.5,35.2,35.9,35.8,35.7
83,"Vancouver, WA",34.6,35.1,36.0,34.8,38.1,36.8,38.0,36.0,36.0,36.3,37.2,36.3,37.1,37.4,36.9,37.1,39.4,36.8,38.1
84,"VA Beach, VA",34.9,35.5,35.8,36.5,34.3,34.9,34.9,34.9,35.0,35.2,35.4,35.8,36.2,36.5,36.6,36.4,37.1,37.8,38.2
85,"Washington, DC",35.9,35.0,34.9,34.9,35.1,33.9,33.4,33.7,33.8,33.8,33.8,33.9,34.0,33.9,34.3,34.1,34.8,34.9,34.9


,Year,City,Total Population,Mean Household Income,Unemployment Rate,Median Age,Geographic Mobility
0,2005,"Irvine city, California",172182,82827.0,27.189537,36.0,170140.0
1,2005,"Long Beach city, California",463956,43746.0,13.441066,33.6,457045.0
2,2005,"Los Angeles city, California",3731437,42667.0,12.035838,33.3,3667157.0
3,2005,"Oakland city, California",373910,44124.0,9.549086,35.2,367165.0
4,2005,"Riverside city, California",294059,50416.0,12.189367,28.9,289333.0
...,...,...,...,...,...,...,...
3500,2023,"Spokane city, Washington",229451,65016.0,3.846681,36.7,226436.0
3501,2023,"Tacoma city, Washington",222932,89107.0,5.968414,37.9,220476.0
3502,2023,"Vancouver city, Washington",196441,80618.0,4.225046,38.1,193173.0
3503,2023,"Madison city, Wisconsin",280307,70484.0,2.634292,32.5,278318.0


In [328]:
pivot_geomob = df.pivot(index='City', columns='Year', values='Geographic Mobility')
df_geomob = pivot_geomob
df_geomob = df_geomob.dropna()
# cleaning out stuff
df_geomob.reset_index(inplace=True)
df_geomob = df_geomob[~df_geomob['City'].str.contains("Puerto Rico", case=False, na=False)]
df_geomob['City'] = df_geomob['City'].str.replace(r' \bcity\b', '', regex=True)
df_geomob['City'] = df_geomob['City'].str.replace(r' \bmunicipality\b', '', regex=True)
df_geomob['City'] = df_geomob['City'].str.replace(r' \bCDP\b', '', regex=True)
df_geomob['City'] = df_geomob['City'].str.strip()
df_geomob['City'] = df_geomob['City'].apply(replace_state_names)
df_geomob.reset_index(inplace=True)
df_geomob = df_geomob.drop(columns=['index'], errors='ignore')
df_geomob

Year,City,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,"Albuquerque, NM",482442.0,485982.0,505490.0,506706.0,519532.0,540574.0,544579.0,548340.0,549629.0,550317.0,552275.0,553849.0,551049.0,553759.0,553804.0,554784.0,558997.0,554890.0,555391.0
1,"Anaheim, CA",323931.0,337196.0,337307.0,324364.0,331183.0,333097.0,337761.0,338945.0,341682.0,342554.0,345316.0,348526.0,348339.0,347902.0,346513.0,349627.0,341397.0,339733.0,337347.0
2,"Anchorage, AK",262394.0,273891.0,275489.0,274493.0,281658.0,287257.0,291302.0,294856.0,297153.0,295739.0,293923.0,294284.0,290742.0,287322.0,283494.0,288241.0,285328.0,282911.0,282455.0
3,"Arlington, VA",188518.0,197518.0,201479.0,207456.0,214759.0,207215.0,212797.0,217781.0,222778.0,223171.0,225805.0,227423.0,231642.0,235053.0,234447.0,233778.0,230062.0,231368.0,231669.0
4,"Arlington, TX",343008.0,350919.0,353799.0,347097.0,374144.0,362376.0,368022.0,369930.0,373284.0,378644.0,382985.0,387022.0,390996.0,392840.0,393950.0,392033.0,388556.0,390590.0,392803.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,"Tulsa, OK",365149.0,375773.0,384454.0,383487.0,383104.0,388405.0,390146.0,388349.0,392521.0,394069.0,396815.0,398007.0,396452.0,396675.0,396163.0,397188.0,406721.0,407633.0,405937.0
83,"Vancouver, WA",153557.0,156168.0,154257.0,164519.0,163418.0,160519.0,163356.0,163438.0,164431.0,167021.0,171059.0,172257.0,173057.0,180893.0,181588.0,180199.0,190718.0,193170.0,193173.0
84,"VA Beach, VA",424961.0,428882.0,427861.0,428105.0,427625.0,433881.0,435581.0,441402.0,442863.0,444498.0,445849.0,446746.0,444913.0,445007.0,443674.0,445525.0,452133.0,449763.0,450178.0
85,"Washington, DC",505886.0,572680.0,579800.0,584439.0,592109.0,596747.0,611608.0,624847.0,636535.0,648586.0,663006.0,672022.0,684810.0,693798.0,697556.0,693253.0,661026.0,660942.0,670127.0


In [345]:
houseprices = pd.read_csv("../data/house_price.csv") 
housedata = houseprices[["RegionName", "2005-01-31","2006-01-31","2007-01-31", "2008-01-31", "2009-01-31", "2010-01-31", "2011-01-31", "2012-01-31", "2013-01-31", "2014-01-31", "2015-01-31", "2016-01-31", "2017-01-31", "2018-01-31", "2019-01-31", "2020-01-31", "2020-01-31", "2021-01-31", "2022-01-31", "2023-01-31"]]
housedata = housedata.sort_values(by='RegionName')
uniquecities = df_unem["City"].unique()
housedata = housedata[housedata["RegionName"].isin(uniquecities)]
housedata.reset_index(inplace=True)
housedata= housedata.drop(columns=['index'], errors='ignore')
housedata.columns = [col[:4] for col in housedata.columns]
housedata.rename(columns={'Regi': 'City'}, inplace=True)
housedata = housedata.dropna()
# uniquecities = housedata["City"].unique()
housedata.reset_index(inplace=True)
housedata= housedata.drop(columns=['index'], errors='ignore')

In [346]:
housedata

,City,2005,2006,2007,2008,2009,2010,2011,2012,2013,...,2015,2016,2017,2018,2019,2020,2020,2021,2022,2023
0,"Albuquerque, NM",153134.096654,170259.007619,193305.919721,200215.211429,187486.929321,180903.756614,171329.029725,164757.245024,163877.783060,...,169546.379112,176038.325622,183130.699662,189542.195949,1.980266e+05,2.129678e+05,2.129678e+05,2.410663e+05,2.805876e+05,3.081899e+05
1,"Anchorage, AK",207175.769453,226903.477999,243855.697573,253043.657003,253020.638590,253109.201265,252403.985149,249462.274330,264355.942255,...,281376.335271,292431.681403,293779.661377,295023.183711,2.973159e+05,3.240456e+05,3.240456e+05,3.282797e+05,3.492012e+05,3.637306e+05
2,"Atlanta, GA",179505.707068,186545.601636,192300.481437,191047.167478,171838.828357,152781.210858,139138.933143,125489.469946,131061.115550,...,169023.933257,183060.400400,196831.953236,213850.991653,2.323420e+05,2.448993e+05,2.448993e+05,2.713745e+05,3.335247e+05,3.669642e+05
3,"Austin, TX",186656.878020,192941.385211,199709.993663,207084.238475,201617.906762,198154.577123,192188.468231,190162.573456,201536.716014,...,248561.105165,271392.931472,287958.928699,298873.135564,3.083495e+05,3.223582e+05,3.223582e+05,3.667843e+05,4.954505e+05,4.995716e+05
4,"Baltimore, MD",240788.263853,287923.885227,308939.030866,304359.493716,278858.068520,259095.858290,245870.769189,232832.506267,240968.189706,...,259239.082824,265118.803553,271451.149002,279396.779775,2.869112e+05,2.966322e+05,2.966322e+05,3.193128e+05,3.489373e+05,3.633435e+05
5,"Boise City, ID",162945.736773,186409.778649,206345.542106,206413.261538,183679.969624,158313.829328,138677.775407,131903.353420,153299.471715,...,184594.983994,200005.423522,220266.965912,242832.875921,2.810533e+05,3.120372e+05,3.120372e+05,3.832637e+05,4.932281e+05,4.692484e+05
6,"Boston, MA",382715.968493,398584.115753,377807.712517,362087.048748,335987.747593,329631.904785,328532.876118,314696.995585,329627.266928,...,375171.911811,402685.595129,428359.342497,457286.235943,4.788747e+05,4.908629e+05,4.908629e+05,5.407756e+05,5.902593e+05,6.247166e+05
7,"Charlotte, NC",158158.778134,163575.501056,171928.977411,179513.455323,171800.455853,163543.225698,154086.630300,145167.451962,149210.006802,...,167340.161683,180624.052307,195158.159561,211730.427000,2.262089e+05,2.395141e+05,2.395141e+05,2.703884e+05,3.319213e+05,3.648160e+05
8,"Chicago, IL",225717.587667,246012.887969,255288.262533,250497.663970,223765.066690,201582.842850,186465.252079,170878.653276,167263.694859,...,196689.317741,206636.520325,216358.817461,228031.162400,2.352609e+05,2.401632e+05,2.401632e+05,2.564646e+05,2.813578e+05,2.951014e+05
9,"Cincinnati, OH",152092.496851,156955.395939,159202.015388,156781.664959,149453.601401,145555.410360,141293.057061,134287.134103,135560.421066,...,146468.346256,154387.478017,162246.146702,171235.495281,1.831568e+05,1.945426e+05,1.945426e+05,2.194252e+05,2.440402e+05,2.625134e+05


In [347]:
uniquecities = housedata["City"].unique()
df_income_cr = df_income[df_income["City"].isin(uniquecities)]
df_income_cr.reset_index(inplace=True)
df_income_cr= df_income_cr.drop(columns=['index'], errors='ignore')
df_unem_cr = df_unem[df_unem["City"].isin(uniquecities)]
df_unem_cr.reset_index(inplace=True)
df_unem_cr= df_unem_cr.drop(columns=['index'], errors='ignore')
df_age_cr = df_age[df_age["City"].isin(uniquecities)]
df_age_cr.reset_index(inplace=True)
df_age_cr= df_age_cr.drop(columns=['index'], errors='ignore')
df_geomob_cr = df_geomob[df_geomob["City"].isin(uniquecities)]
df_geomob_cr.reset_index(inplace=True)
df_geomob_cr= df_geomob_cr.drop(columns=['index'], errors='ignore')

In [349]:
df_income_cr.to_csv('../data/income_57cities.csv')
df_unem_cr.to_csv('../data/unemployment_57cities.csv')
df_age_cr.to_csv('../data/medianage_57cities.csv')
df_geomob_cr.to_csv('../data/geomobility_57cities.csv')
housedata.to_csv('../data/houseprice_57cities.csv')

0      Albuquerque, NM
1          Atlanta, GA
2           Austin, TX
3        Baltimore, MD
4           Boston, MA
5          Buffalo, NY
6        Charlotte, NC
7          Chicago, IL
8       Cincinnati, OH
9        Cleveland, OH
10        Columbus, OH
11          Dallas, TX
12          Denver, CO
13         Detroit, MI
14          Fresno, CA
15         Houston, TX
16    Jacksonville, FL
17         Memphis, TN
18           Miami, FL
19       Milwaukee, WI
20     Minneapolis, MN
21           Omaha, NE
22         Orlando, FL
23    Philadelphia, PA
24         Phoenix, AZ
25      Pittsburgh, PA
26        Portland, OR
27         Raleigh, NC
28      Sacramento, CA
29         Seattle, WA
30           Tampa, FL
31          Tucson, AZ
32           Tulsa, OK
33      Washington, DC
34         Wichita, KS
Name: City, dtype: object